In [18]:
from pyspark.sql import functions as F
from service.utils.spark import get_spark_session
spark = get_spark_session(dev=True)

25/09/23 15:54:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [19]:
test_namespace = 'gold' 
# spark.sql(f"DROP NAMESPACE IF EXISTS {test_namespace} CASCADE")
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {test_namespace}").show()

++
||
++
++



In [20]:
tmp_order_product = spark.read.table(f"{test_namespace}.tmp_order_product")

In [21]:
tmp_order_product.show(2)

+--------------------+--------------------+----------+-------------+--------------------+-------------------+-----------+
|            order_id|          product_id|  category|order_item_id|           seller_id|shipping_limit_date|total_price|
+--------------------+--------------------+----------+-------------+--------------------+-------------------+-----------+
|00010242fe8c5a6d1...|4244733e06e7ecb49...|cool_stuff|            1|48436dade18ac8b2b...|2017-09-19 09:45:35|      72.19|
|00018f77f2f0320c5...|e5f2d52b802189ee6...|  pet_shop|            1|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|     259.83|
+--------------------+--------------------+----------+-------------+--------------------+-------------------+-----------+
only showing top 2 rows



In [22]:
tmp_order_timestamp = spark.read.table('warehousedev.gold.tmp_order_timestamp')
tmp_order_timestamp.show(n=5)

+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+
|            order_id|         customer_id|           purchase|           approved|  delivered_carrier| delivered_customer|
+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+
|014405982914c2cde...|e8d87ee946600f775...|2017-07-26 17:38:47|2017-07-26 17:50:17|2017-07-27 19:39:52|2017-07-31 15:53:33|
|019886de8f385a39b...|d29ede26cd3e2817b...|2018-02-10 12:52:51|2018-02-10 13:08:12|2018-02-14 15:28:51|2018-02-23 02:03:03|
|01a6ad782455876aa...|31c23262d79bc7e80...|2018-01-18 10:07:52|2018-01-18 10:17:29|2018-01-22 22:37:04|2018-02-01 21:02:22|
|01d907b3e209269e1...|a7b781410bcc8bbf3...|2017-08-09 16:21:06|2017-08-10 10:25:08|2017-08-11 19:05:53|2017-08-16 22:34:11|
|028dc52e12ddda803...|148a35e3469010c97...|2017-12-19 09:43:54|2017-12-19 10:53:22|2017-12-20 16:49:18|2017-12-21 14:22:56|
+-------

In [23]:
ed_df = spark.read.csv("s3a://warehousedev/bronze/tsv/estimated_delivery_date.tsv", header=True, sep='\t')
format_string = 'yyyy-MM-dd HH:mm:ss'
ed_df = ed_df.withColumn('estimated_delivery_date', F.to_timestamp('estimated_delivery_date', format_string))

In [24]:
shipping_limit_date = \
    tmp_order_product.select('order_id', 'shipping_limit_date').withColumn('shipping_limit_date', F.to_timestamp('shipping_limit_date', format_string))


In [25]:
delivered_order_timestamp = shipping_limit_date.join(tmp_order_timestamp, on='order_id', how='outer')

In [26]:
delivered_order_timestamp = delivered_order_timestamp.join(ed_df, on='order_id', how='outer')

In [27]:
delivered_order_timestamp = delivered_order_timestamp.dropna().drop_duplicates()

In [28]:
delivered_order_timestamp.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- purchase: timestamp (nullable = true)
 |-- approved: timestamp (nullable = true)
 |-- delivered_carrier: timestamp (nullable = true)
 |-- delivered_customer: timestamp (nullable = true)
 |-- estimated_delivery_date: timestamp (nullable = true)



In [29]:
delivered_order_timestamp.writeTo(f'{test_namespace}.delivered_order_timestamp').createOrReplace()

In [30]:
spark.stop()